# The Condorcet-Schulze method

The Schulze method is a refinemnet of the original Condorcet methods that are used for deciding the winning candidate of an election in which all the voters can express a number of different preferences.

To read all about the method we have been using, have a look at [Wikipedia](https://en.wikipedia.org/wiki/Schulze_method).

## How to use the condorcet module

Create a dataframe with the ordered preferences for each voter. The first column shall be the first preference for the i-th voter.

The entries of each cell are the names of the candidates.

Each voter can vote for all or part of the candidates. The non voted candidates are considered to be less preferred than the voted ones.


In [6]:
"""Ranks candidates by the Condorcet method with also unvoted candidates.

For more information, please refer to https://en.wikipedia.org/wiki/Condorcet_method.
"""

__author__ = "Matteo Caorsi"
## I thank Michael G. Parker (http://omgitsmgp.com/) from whom I took the code skeleton

import numpy as np
from collections import defaultdict

def candidate_names_from_df(df):
    return list(np.unique(df.values.flatten()))

def weighted_ranks_from_df(df):
    weighted_ranks = []
    index = 0
    for row in df.values:
        weighted_ranks.append((list(row), 1))
        index += 1
    return weighted_ranks


def _add_remaining_ranks(d, candidate_name, remaining_ranks, weight):
    for other_candidate_name in remaining_ranks:
        d[candidate_name, other_candidate_name] += weight


def _add_ranks_to_d(d, ranks, weight, unvoted_candidates):
    for i, candidate_name in enumerate(ranks):
        remaining_ranks = ranks[i+1:] + unvoted_candidates
        _add_remaining_ranks(d, candidate_name, remaining_ranks, weight)


def _compute_d(weighted_ranks, candidate_names):
    """Computes the d array in the Schulze method.

        d[V,W] is the number of voters who prefer candidate V over W.

        We consider unvoted candidates as being ranked less than any
        other candidate voted by the voter.
        """
    d = defaultdict(int)
    for ranks, weight in weighted_ranks:
        unvoted_candidates = list(set(candidate_names)-set(ranks))
        #print("unoted:", unvoted_candidates)
        _add_ranks_to_d(d, ranks, weight, unvoted_candidates)
    #print(d)
    return d


def _compute_p(d, candidate_names):
    '''Computes the p array in the Schulze method.

        p[V,W] is the strength of the strongest path from candidate V to W.
    '''

    # taken directly from wikipedia: https://en.wikipedia.org/wiki/Schulze_method#Implementation
    p = {}
    for candidate_name1 in candidate_names:
        for candidate_name2 in candidate_names:
            if candidate_name1 != candidate_name2:
                # get the value from the d matrix or default it to 0
                strength = d.get((candidate_name1, candidate_name2), 0)
                if strength > d.get((candidate_name2, candidate_name1), 0):
                    p[candidate_name1, candidate_name2] = strength
                else:
                    p[candidate_name1, candidate_name2] = 0

    for candidate_name1 in candidate_names:
        for candidate_name2 in candidate_names:
            if candidate_name1 != candidate_name2:
                for candidate_name3 in candidate_names:
                    if (candidate_name1 != candidate_name3) and (candidate_name2 != candidate_name3):
                        curr_value = p.get((candidate_name2, candidate_name3), 0)
                        new_value = min(
                            p.get((candidate_name2, candidate_name1), 0),
                            p.get((candidate_name1, candidate_name3), 0))
                        p[candidate_name2, candidate_name3] = max(curr_value,new_value)
    return p


def _rank_p(candidate_names, p):
    """Ranks the candidates by p."""
    # how many times does a candidate wins against each of the others?
    candidate_wins = defaultdict(list)

    for candidate_name1 in candidate_names:
        num_wins = 0

        # Compute the number of wins this candidate has over all other candidates.
        for candidate_name2 in candidate_names:
            if candidate_name1 != candidate_name2:
                candidate1_score = p.get((candidate_name1, candidate_name2), 0)
                candidate2_score = p.get((candidate_name2, candidate_name1), 0)
                if candidate1_score > candidate2_score:
                    num_wins += 1

        candidate_wins[num_wins].append(candidate_name1)
        #print(candidate_wins)
    sorted_wins = sorted(candidate_wins.keys(), reverse=True)
    return [candidate_wins[num_wins] for num_wins in sorted_wins]


def compute_ranks(df):
    weighted_ranks = weighted_ranks_from_df(df)
    candidate_names = candidate_names_from_df(df)
    #print(candidate_names)
    d = _compute_d(weighted_ranks, candidate_names)
    p = _compute_p(d, candidate_names)
    return _rank_p(candidate_names, p)

In [7]:
import numpy as np
import pandas as pd

# Load the submissions to merge

In [8]:
import re

#load submissions
submissions = [] #files
submissions_path = [
    "data/submissions/condorcet_sub (1).csv",
    "data/submissions/condorcet_sub (2).csv",
    "data/submissions/condorcet_sub (3).csv",
    "data/submissions/condorcet_sub (4).csv",

]
submissions_lines = [] #lines

#read lines of each submission
for i, path in enumerate(submissions_path):
    submissions.append(open(path, 'r'))
    submissions_lines.append(submissions[i].readlines())

sub_len = len(submissions_lines[0])

#create final submission file
new_sub = open('data/submission_CondorcetSchulze.csv', 'w')
new_sub.write('user_id,item_list\n')

18

In [9]:
def unfold_list(list):
    recommendations = []
    for position in ranking:
        for item in position:
            recommendations.append(item)
    del recommendations[10:]
    return recommendations

In [10]:
#iterate through each line
for i in range(1, sub_len):
    #print("Reading line {}".format(i))
    user = 0
    recommendations_df = pd.DataFrame(columns=['r0', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9'])
    #iterate through each submission
    for o in range(len(submissions)):
        #print("Of recommender {}".format(o))
        submission_line = submissions_lines[o][i]
        #the line is formatted as "user, rec1 rec2 rec3 ... rec10"
        #but sometimes there are more spaces (a tab perhaps?)
        #regex to remove multiple spaces
        submission_line = re.sub(' +', ' ', submission_line)
        #regex to remove new line
        submission_line = re.sub('\n', '', submission_line)
        #regex to remove the very first space
        submission_line = re.sub(', ', ',', submission_line)
        #we split the line to separate the user_id from the recommendations
        submission_line_params = submission_line.split(sep=',')
        #saving the user id
        user = submission_line_params[0]
        #getting the recommends as lists
        recommendations_df.loc[len(recommendations_df)] = submission_line_params[1].split(' ')
    ranking = compute_ranks(recommendations_df)
    ranking = unfold_list(ranking)
    #write line
    new_rec_line = user + ", " + " ".join(ranking) + "\n"
    new_sub.write(new_rec_line)

In [11]:
new_sub.close()
for submission in submissions:
    submission.close()